# USGS TNW API

The national map is a collection of topological datasets, maintained by the USGS. 

It provides an API endpoint which can be used to find downloadable links for the products offered.
- Full description of datasets available can retrieved.
  This consists of metadata such as detail description and publication dates.
- A wide range of dataformats are availble

More complete documentation for the API can be found at
https://apps.nationalmap.gov/tnmaccess/#/



In [1]:
import requests
import json

class The_national_map_USGS():
    """
    The national map is a collection of topological datasets, maintained by the USGS. 

    It provides an API endpoint which can be used to find downloadable links for the products offered.
        - Full description of datasets available can retrieved.
          This consists of metadata such as detail description and publication dates.
        - A wide range of dataformats are availble

    More complete documentation for the API can be found at
        https://apps.nationalmap.gov/tnmaccess/#/
    """

    def __init__(self):
        self.api_endpoint = r'https://tnmaccess.nationalmap.gov/api/v1/'
        self.DS = self.datasets_full()
    
    def datasets_full(self):
        """
        Full description of datasets provided.
        Returns a JSON or empty list.
        """
        try:
            return requests.get(f'{self.api_endpoint}datasets?').json()
        except:
            print('Failed to load metadata from The National Map API endpoint V1')
            return []

    @property
    def prodFormats(self):
        """
        Return all datatypes available in any of the collections. 
        Note that "All" is only peculiar to one dataset. 
        """
        return set(i['displayName'] for ds in self.DS for i in ds['formats'])

    @property
    def datasets(self):
        """
        Returns a list of dataset tags (most common human readable self description for specific datasets).
        """
        return set(y['sbDatasetTag'] for x in self.DS for y in x['tags'])

    def find_tiles(self, 
                   bbox:list[float] = None, 
                   polygon:list[tuple[float,float]] = None, 
                   datasets:list[str] | str = [], 
                   prodFormats:list[str] | str = [],
                   prodExtents:list[str] | str = [], 
                   q:str = None, 
                   dateType:str = None, 
                   start:str = None, 
                   end:str = None, 
                   offset:int = 0, 
                   max:int = None, 
                   outputFormat:str = 'JSON', 
                   polyType:str = None, 
                   polyCode:str = None, 
                   extentQuery:int = None) -> dict:
        """
        Possible search parameters (kwargs) support by API

        Parameter               Values                      
            Description
        ---------------------------------------------------------------------------------------------------    
        bbox                    'minx, miny, maxx, maxy'
            Geographic longitude/latitude values expressed in  decimal degrees in a comma-delimited list.
        polygon                 '[x,y x,y x,y x,y x,y]'       
            Polygon, longitude/latitude values expressed in decimal degrees in a space-delimited list.
        datasets                See: Datasets (Optional)       
            Comma-delimited list of valid dataset tag names (sbDatasetTag)
            From https://apps.nationalmap.gov/tnmaccess/#/product
        prodFormats             See: Product Formats (Optional)
            Comma-delimited list of dataset-specific formats
            From https://apps.nationalmap.gov/tnmaccess/#/product
        prodExtents             See: Product Extents (Optional)
            Comma-delimited list of dataset-specific extents
            From https://apps.nationalmap.gov/tnmaccess/#/product
        q                       free text 
            Text input which can be used to filter by product titles and text descriptions.
        dateType                dateCreated | lastUpdated | Publication 
            Type of date to search by.
        start                   'YYYY-MM-DD' 
            Start date
        end                     'YYYY-MM-DD' 
            End date (required if start date is provided)
        offset                  integer 
            Offset into paginated results - default=0
        max                     integer 
            Number of results returned
        outputFormat            JSON | CSV | pjson
            Default=JSON
        polyType                state | huc2 | huc4 | huc8 
            Well Known Polygon Type. Use this parameter to deliver data by state or HUC
            (hydrologic unit codes defined by the Watershed Boundary Dataset/WBD)
        polyCode                state FIPS code or huc number 
            Well Known Polygon Code. This value needs to coordinate with the polyType parameter.
        extentQuery             integer 
            A Polygon code in the science base system, typically from an uploaded shapefile
        """

       
        # call locals before creating new locals
        used_locals = {k:v for k,v in locals().items() if v and k != 'self'}

        # Parsing
    
        def convert_polygon(x):
            return ','.join(' '.join(map(str,point)) for point in x)
        if polygon:
            used_locals['polygon'] = convert_polygon(polygon)        
        
        # Partial validation
        # Fetch list seems broken in API ???, only takes list with 1 item or str.
        # Could be map AND instead of OR for processing list

        assert set(datasets).issubset(self.datasets) or datasets in self.datasets, f'Unknown datasets, must be elements of {self.datasets}'
        assert set(prodFormats).issubset(self.prodFormats) or prodFormats in self.prodFormats, f'Unknown prodFormats, must be element of {self.prodFormats}'

        # Validations handled better (f.e. psjon) by API endpoint error responses

        '''
        import datetime
        def validate_date(date_text):
            try:
                datetime.datetime.strptime(date_text, '%Y-%m-%d')
                return True
            except ValueError:
                return False

        can = {'JSON', 'CSV'} # psjon added
        assert not outputFormat or outputFormat in can, f'Unknown outputFormat, must be element of {can}'
        can = {'dateCreated', 'lastUpdated', 'Publication'}
        assert not dateType or dateType in can, f'Unknown dataType, must be element of {can}'       
        if start or end or dateType:
            assert start and end and dateType and validate_date(start) and validate_date(end), """
            Argument 'start', 'end' and 'dateType' should be used together, 
            and 'start', 'end' should be formatted as YYYY-MM-DD"""
        '''
            
        # Fetch response

        response = requests.get(f'{self.api_endpoint}products?', params=used_locals)
        if response.status_code//100 == 2:
            return response.json()
        else:
            print(response.json())
        return {}

## Example usage

In [2]:
U = The_national_map_USGS()

In [3]:
U.datasets

{'Alaska IFSAR 5 meter DEM',
 'Digital Elevation Model (DEM) 1 meter',
 'Ifsar Digital Surface Model (DSM)',
 'Ifsar Orthorectified Radar Image (ORI)',
 'Lidar Point Cloud (LPC)',
 'National Elevation Dataset (NED) 1 arc-second',
 'National Elevation Dataset (NED) 1/3 arc-second',
 'National Elevation Dataset (NED) 1/3 arc-second - Contours',
 'National Elevation Dataset (NED) 1/9 arc-second',
 'National Elevation Dataset (NED) Alaska 2 arc-second',
 'National Hydrography Dataset (NHD) Best Resolution',
 'National Hydrography Dataset Plus High Resolution (NHDPlus HR)',
 'National Watershed Boundary Dataset (WBD)',
 'Original Product Resolution (OPR) Digital Elevation Model (DEM)',
 'Small-scale Datasets - Boundaries',
 'Small-scale Datasets - Contours',
 'Small-scale Datasets - Hydrography',
 'Small-scale Datasets - Transportation',
 'Topobathymetric Lidar DEM',
 'Topobathymetric Lidar Point Cloud',
 'US Topo Current',
 'US Topo Historical'}

In [4]:
U.prodFormats

{'All',
 'FileGDB',
 'GeoPDF',
 'GeoPackage',
 'GeoTIFF',
 'GeoTIFF, IMG',
 'JPEG2000',
 'LAS,LAZ',
 'Shapefile',
 'TIFF',
 'TXT (pipes)'}

## API error handling

In [5]:
U.find_tiles(start='2021-01-01',end='2022-31-01',q='NED',outputFormat='JSON')

{'errorMessage': "[BadRequest] 'Value '2022-31-01' of property end is not a valid date (YYYY-MM-DD)' "}


{}

In [6]:
U.find_tiles(start='2021-01-01',end='2022-01-01',q='NED',outputFormat='JSON')

{'errorMessage': "[BadRequest] 'date type is required when start date and enddate are provided' "}


{}

API error handling is better than the API docs (pjson not mentioned as possible format)

In [7]:
U.find_tiles(start='2021-01-01',end='2022-01-01',dateType='dateCreated',q='NED',outputFormat='XLS')

{'errorMessage': "[BadRequest] 'Value 'XLS' of property outputFormat must be csv, pjson, or json' "}


{}

## Get results

Provides detailed meta data regarding tiles.

- moreInfo: Short description of the dataset
- metaUrl: describing the dataset properties as human readable
- vendorMetaUrl: describing the dataset properties as XML
- downloadURL: download link to actual file
- And many others (see below)
- Available key-values can vary depending on dateset.

In [8]:
paras = {'prodFormats':'LAS,LAZ', 
         'datasets':['Lidar Point Cloud (LPC)'],
         'polygon': [
                     (-104.94262695312236, 41.52867510196275),
                     (-102.83325195312291, 41.52867510196275),
                     (-102.83325195312291, 40.45065268246805),
                     (-104.94262695312236, 40.45065268246805),
                     (-104.94262695312236, 41.52867510196275),
                   ]
        }

U.find_tiles(**paras)['items'][0]

{'title': 'USGS Lidar Point Cloud CO SoPlatteRiver-Lot1 2013 13TFE627477 LAS 2015',
 'moreInfo': 'Lidar (Light detection and ranging) discrete-return point cloud data are available in the American Society for Photogrammetry and Remote Sensing (ASPRS) LAS format. The LAS format is a standardized binary format for storing 3-dimensional point cloud data and point attributes along with header information and variable length records specific to the data. Millions of data points are stored as a 3-dimensional data cloud as a series of x (longitude), y (latitude) and z (elevation) points. A few older projects in this collection are in ASCII format. Please refer to http://www.asprs.org/Committee-General/LASer-LAS-File-Format-Exchange-Activities.html for additional information.',
 'sourceId': '5a801f46e4b00f54eb2a10dc',
 'sourceName': 'ScienceBase',
 'sourceOriginId': None,
 'sourceOriginName': 'gda',
 'metaUrl': 'https://www.sciencebase.gov/catalog/item/5a801f46e4b00f54eb2a10dc',
 'vendorMetaUr

In [9]:
paras = {
         'datasets':'National Elevation Dataset (NED) 1/3 arc-second',
         'polyCode': '01010002',
         'polyType': 'huc8'
        }

U.find_tiles(**paras)['items'][0]

{'title': 'USGS 1/3 Arc Second n47w069 20210611',
 'moreInfo': 'This tile of the 3D Elevation Program (3DEP) seamless products is 1/3 Arc Second resolution. 3DEP data serve as the elevation layer of The National Map, and provide basic elevation information for Earth science studies and mapping applications in the United States. Scientists and resource managers use 3DEP data for global change research, hydrologic modeling, resource monitoring, mapping and visualization, and many other applications. 3DEP data compose an elevation dataset that consists of seamless layers and a high resolution layer. Each of these layers consists of the best available raster elevation data of the conterminous United States, Alaska, Hawaii, territorial islands, Mexico and Canada. 3DEP data are updated continually as [...]',
 'sourceId': '60d2c059d34e840986528938',
 'sourceName': 'ScienceBase',
 'sourceOriginId': None,
 'sourceOriginName': 'gda',
 'metaUrl': 'https://www.sciencebase.gov/catalog/item/60d2c059